# KaxaNuk Data Corrector: error analysis

## 00: Preamble
Data provider issues are a rather common problem that financial analysts need to address. Data quality translates to model quality, model quality translates into alpha. The purpose of this challenge is to first gain an understanding of the problems by analyzing the problems themselves, and their underlying causes and patterns. This in order to propose better informed solutions to data correction or imputation.
Once the error is identified, an appropriate imputation method should be implemented. This approach explores deterministic, statistical and machine-learning error detection and imputation. Regardless of the methodology, error correction in financial data should always be accompanied by an error log forensic analysis in order to minimize false positives. The forensic analysis in this work was done using a large language model (claude opus 4.5) to scrape financial news sites for events that may be related to the observed error logs. After thoroughly examining the forensic analysis, false positive flagging should always be under the practicioner's criteria.

## 01: Provided error log analysis

Financial Modeling Prep's data errors are **not isolated incidents but systemic issues** concentrated in three categories: non-standard securities (preferred shares, warrants, senior notes), companies undergoing corporate actions, and illiquid micro-cap stocks. The errors stem from data architecture that struggles with securities that deviate from standard common stock data structures.

### The "sorted by date" errors

The 74 tickers generating "FundamentalData.rows not correctly sorted by date" errors reveal teh same patterns. Nearly **40% are non-common equity securities**—preferred shares, warrants, or senior notes—that have fundamentally different data reporting requirements than common stock. The B. Riley Financial family alone contributes 8 tickers (RILY, RILYG, RILYK, RILYL, RILYN, RILYT, RILYZ, RILYP), spanning common stock, preferred shares, and tradeable senior notes. Federal Agricultural Mortgage (Farmer Mac) adds 8 more (AGM and seven preferred series), while Presidio Property Trust contributes common stock, preferred shares, and warrants.

**Corporate actions create data discontinuities** across this list. At least 12 tickers underwent mergers, acquisitions, or name changes in 2024-2025:

| Ticker | Event | Date |
|--------|-------|------|
| FARO | Acquired by AMETEK | July 2025 |
| IVAC | Acquired by Seagate | March 2025 |
| SASR | Acquired by Atlantic Union | April 2025 |
| APDN | Rebranded to BNBX | October 2025 |
| MICS | Became RIME | September 2024 |
| ATON | Rebranded AlphaTON Capital | September 2025 |
| NBP | Former I-Mab, now NovaBridge | October 2025 |

Several companies are in financial distress: B. Riley Financial suspended dividends and faces Nasdaq delisting risk after **435-475M quarterly losses**; Ideanomics (IDEX) filed Chapter 11 bankruptcy in December 2024 following SEC fraud settlements; Staffing 360 Solutions (STAF) was delisted to OTC.

### Market cap distribution

The market cap breakdown exposes another pattern: **approximately 35-40% of affected tickers are micro-cap stocks** (under 300M market capitalization). These include XELB (8-11M), EVTV (20M), SOTK (40M), TPCS (35M), and DLPN ($50M). Micro-cap stocks typically have less rigorous data reporting, lower analyst coverage, and more frequent data quality issues due to limited institutional oversight.

| Market Cap Category | Percentage of Error Tickers |
|---------------------|---------------------------|
| Micro-cap (<$300M) | ~35-40% |
| Small-cap ($300M-$2B) | ~25-30% |
| Mid-cap ($2B-$10B) | ~20% |
| Large-cap (>$10B) | ~15% |

The large-cap tickers that appear—DOV (27B), DG (22-24B), JBL (18B), RBA (19.5B)—likely experience errors due to corporate actions rather than data quality. RBA (RB Global) completed a major merger with IAA in 2023, and MTZ (MasTec) faced shareholder lawsuits creating reporting complexities.

### Preferred shares dominate the "no data returned" errors

All 12 tickers generating "No data returned by unadjusted market data endpoint" errors are **preferred shares or eliminated share classes**:

- **PEI series** (PEI-PB, PEI-PC, PEI-PD): Pennsylvania REIT preferred shares—company emerged from Chapter 11 bankruptcy in 2020 with restructured capital
- **PSB series** (PSB-PX, PSB-PY, PSB-PZ): PS Business Parks preferred depositary shares—parent company was acquired by Blackstone for 7.6 billion in 2022, delisting common stock but potentially leaving preferred shares trading
- **NRZ series** (NRZ-PA, NRZ-PB, NRZ-PC): Rithm Capital (formerly New Residential Investment) fixed-to-floating rate preferreds experiencing LIBOR transition complications
- **STZ-B**: Constellation Brands Class B stock—**eliminated entirely in November 2022** when the Sands family exchanged their super-voting shares for $64.64 cash plus Class A shares
- **PNC-PP**: PNC Financial Series P preferred with complex fixed-to-floating rate structure
- **ALP-PQ**: Appears to be an invalid or delisted ticker

The pattern is unmistakable: FMP's unadjusted market data endpoint cannot handle preferred share structures, depositary shares, or securities that no longer trade but retain historical data.

### Warrants create impossible price relationships

The three tickers with "MarketDataDailyRow low > high" errors are all **SPAC warrants trading at near-zero prices**:

| Ticker | Company | Current Price | Status |
|--------|---------|---------------|--------|
| UWMC-WT | UWM Holdings | ~$0.01 | NYSE delisting proceedings initiated December 19, 2025 |
| BFLY-WT | Butterfly Network | ~$0.02 | Extremely illiquid, ~44K average daily volume |
| ML-WT | MoneyLion | ~$0.26 | Thinly traded, expires September 2026 |

When securities trade at fractions of a penny with minimal volume, bad tick data becomes inevitable. Wide bid-ask spreads, stale quotes, and erroneous trade reports create situations where recorded daily lows can exceed daily highs. UWMC-WT is actively being delisted for "abnormally low selling price"—the security is essentially worthless.

### Negative shares outstanding traces to corporate restructuring

The three "Negative shares outstanding" errors (HELE, QLGN, ELDN) correlate directly with significant corporate events:

**HELE** (Helen of Troy, error date May 1, 2017): No stock splits, but the company operates on a February fiscal year-end. The error date falls during fiscal year transitions when share counts from buyback programs may create calculation discrepancies across FMP's data sources.

**QLGN** (Qualigen Therapeutics, error date November 14, 2025): This company has undergone **two reverse stock splits** (1-for-10 in 2022, 1-for-50 in 2024), was acquired by Faraday Future as a 55% stakeholder, and rebranded to AIxCrypto Holdings in November 2025—all creating massive data discontinuities.

**ELDN** (Eledon Pharmaceuticals, error date November 14, 2025): A **$50 million dilutive offering** closed around November 12, 2025, adding 15+ million shares plus warrants, increasing share count by over 100% year-over-year. The error date coincides exactly with this offering.

### The single negative price error points to data corruption

ASB-PF (Associated Banc-Corp Series F Preferred) showing a negative low price is simply **data corruption**. Preferred stocks have complex ex-dividend adjustments, and a calculation error in FMP's dividend adjustment pipeline likely produced an impossible negative value. The security trades normally around $21 with a 6.65% yield.

### Conclusions: systemic issues with non-standard securities

These errors demonstrate FMP has **architectural limitations handling three categories of securities**:

**Non-standard security types**: Preferred shares, warrants, senior notes, and depositary shares have different data structures, reporting requirements, and pricing mechanics than common stock. FMP's fundamental data infrastructure appears designed primarily for common equity.

**Corporate action transitions**: Mergers, acquisitions, reverse splits, name changes, and bankruptcies create data discontinuities. When Constellation Brands eliminated STZ-B or Blackstone acquired PSB, historical data must be handled differently—FMP's pipeline struggles with these transitions.

**Illiquid and penny securities**: When UWMC-WT trades at $0.0098 with minimal volume, standard data validation breaks down. The "low > high" errors are essentially the data provider acknowledging bad tick data from nearly untradeable securities.

The pattern suggests it would be beneficial to disable the native lock the data curator has that yielded this error log, and implement robust handling taking into account non-common equity and apply different validation rules capable of handling illiquid instruments while logging the errors detected. For users, these errors serve as a useful audit tool to identify overall quality of a data provider.




## 02: Sanity check

The previous log analysis and data correction strategies were consequent to a thorough forensic examination of the critical errors that kept the datacurator from extracting the data. This next section documents the functions used to detect and correct error types found in the set of 6000+ tickers, including the errors found in the log and those that did not yield an error that prevented DataCurator from downloading the data. Still, financial logic dictates the encountered values should not be possible, so these values were all treated with a deterministic filter and imputation strategy.

These functions act as a post-download validation and correction layer. The data arrives intact but contains logical inconsistencies, impossible values, or violations of financial accounting identities. Left uncorrected, these issues propagate through backtests, valuations, and risk models—often without triggering obvious failures.

The functions below implement deterministic, auditable corrections with full logging of every modification made to the source data. These functions could be implemented as custom calculation within the DataCurator module (without explicit error logging) or by separate within the EDA module (with explicit error logging).

---

### 1. sort_dates

#### Error Addressed
**Out-of-order fundamental data rows.** Financial statements may arrive with dates that are not chronologically sorted due to:
- Amended filings (10-K/A, 10-Q/A) inserted after original statements
- Fiscal year-end changes creating overlapping periods
- Data provider ingestion timing mismatches

#### Approach
0. Edit the Data Curatos so the dates do not need to be sorted for data to be downloaded. Ingest the uncurated data.
1. Establish a date hierarchy: primary date column (`m_date`) → filing date (`f_filing_date`)
2. Preserve null-dated rows in their original positions (they cannot be sorted)
3. Sort only valid-dated rows while maintaining relative positions
4. Deduplicate by keeping earliest or latest filing per period
5. Log all position changes for audit

#### Correction Method
- **Reordering only**—no values are modified
- Null rows remain untouched
- Deduplication uses filing date to determine which version to keep

---

### 2. fill_negatives_fundamentals

#### Error Addressed
**Negative values in fundamental data columns where negatives are impossible.** Examples include:
- Negative shares outstanding
- Negative total assets
- Negative revenue (in contexts where it should be gross revenue)

These typically result from data entry errors, sign convention mismatches, or incorrect aggregation.

#### Approach
1. Scan specified columns for values < 0
2. Replace negatives with `NULL`
3. Apply forward-fill to propagate the last valid (non-negative) value
4. Log every replaced value with ticker, date, and original value

#### Correction Method
- **Forward-fill from last known good value**
- Assumes temporal continuity: if shares outstanding was 100M yesterday and shows -5M today, yesterday's value is more trustworthy
- Since fundamental data follows a stepwise function, interpolation would introduce values which never existed. This is why forward-fill is preferred.

---

### 3. fill_negatives_market

#### Error Addressed
**Negative prices in market data.** A stock price cannot be negative, but data corruption from:
- Dividend adjustment calculation errors
- Bad tick data propagation
- Corporate action misapplication

can produce impossible negative values in OHLC or VWAP columns.

#### Approach
1. Identify negative values in specified columns
2. Gather up to 4 previous valid (non-negative) data points
3. Fit a **backward-looking cubic spline** to extrapolate a replacement value
4. If fewer than 3 prior points exist, fall back to last valid value (if no such value exists, default to zero)
5. If spline produces negative or non-finite result, fall back to last valid value

#### Correction Method
- **Cubic spline interpolation (backward-looking only)**
- Explicitly avoids look-forward bias for backtesting integrity
- Preserves original null positions
- Falls back gracefully when insufficient history exists

---

### 4. zero_wipeout

#### Error Addressed
**Zero values in share-related columns when trading volume is positive.** This paradox indicates data corruption:
- If volume > 0, trading occurred
- If shares outstanding = 0, the company has no equity
- Both cannot be true simultaneously

Common cause: placeholder zeros inserted during data pipeline failures.

#### Approach
1. Identify rows where ANY target column equals 0 AND `m_volume` > 0
2. Replace the zero with `NULL`
3. Apply forward-fill to restore continuity
4. Log all affected rows

#### Correction Method
- **Conditional forward-fill**
- Only triggers when the logical impossibility (zero shares + positive volume) is detected
- Leaves legitimate zeros (pre-IPO, delisted) untouched if volume is also zero

---

### 5. mkt_cap_scale_error

#### Error Addressed
**10x or greater jumps in market cap or shares outstanding.** These typically indicate:
- Unit conversion errors (shares in units vs. thousands vs. millions)
- Data source switches mid-series
- Incorrect corporate action adjustments

The 10x jump parameter was arbitrarily chosen, this value should be adjusted based on the investment universe: it is impossible for large caps to jump 5x overnight (largest jump in history was Volkswagen in 2008 with a ~4x jump), but in small caps it is a reasonable outlier. Placing a 10x threshold on a universe that contains only large caps would yield false negatives. A reasonable generalized approach is a statistical filter (see section 4).

#### Approach
1. Compare each row's value to the previous row
2. Flag rows where value ≥ 10× prior value
3. Detect **correlated jumps**: if both market cap AND shares outstanding jump together, the error likely spans multiple rows
4. For correlated jumps, identify the entire error span (values within 20% of jumped value)
5. Apply forward-fill to replace the corrupted span

#### Correction Method
- **Forward-fill with span detection**
- Single-row spikes: replace with prior value
- Multi-row plateaus: identify the elevated region and replace entirely
- Logs include error type classification

---

### 6. ohlc_integrity

#### Error Addressed
**Violations of OHLC mathematical constraints:**
- High < max(Open, Close, Low) → High should be the maximum
- Low > min(Open, Close, High) → Low should be the minimum
- VWAP outside [Low, High] → VWAP must fall within the day's range

These violations break technical indicators, volatility calculations, and charting.

#### Approach
Validates three column groups independently:
1. Raw OHLC (`m_open`, `m_high`, `m_low`, `m_close`, `m_vwap`)
2. Split-adjusted OHLC
3. Dividend-and-split-adjusted OHLC

For each group:
- Compute actual max/min of OHLC values
- Compare against declared High/Low
- Check VWAP bounds

#### Correction Method
| Violation | Correction |
|-----------|------------|
| High < actual max | Set High = max(O, H, L, C) |
| Low > actual min | Set Low = min(O, H, L, C) |
| VWAP outside range | Set VWAP = (O + H + L + C) / 4 |

- Uses OHLC centroid as VWAP replacement (simple average, not volume-weighted, but mathematically valid)

---

### 7. validate_financial_equivalencies

#### Error Addressed
**Violations of fundamental accounting identities:**

**Hard Filters (corrected):**
- Assets ≠ Current Assets + Noncurrent Assets
- Liabilities ≠ Current Liabilities + Noncurrent Liabilities

**Soft Filters (flagged only):**
- Stockholder Equity ≠ Common Stock + APIC + Retained Earnings + Other Equity
- Period End Cash ≠ Cash and Cash Equivalents
- Assets ≠ Liabilities + Equity + Noncontrolling Interest

#### Approach
1. Compute component sums for each identity
2. Compare against declared totals with configurable tolerance (default 5%)
3. For hard filters: apply proportional scaling to force balance
4. For soft filters: set `data_warning` flag without modifying values

#### Correction Method (Hard Filters)
**Proportional Scaling:**
```
Factor = Total / (Current + Noncurrent)
Corrected_Current = Current × Factor
Corrected_Noncurrent = Noncurrent × Factor
```

**Edge Case:** If components sum to 0 but total ≠ 0, the entire total is assigned to the noncurrent bucket (residual plug).

Soft filter violations are logged but not corrected because:
- Equity components may have legitimate "other" buckets not captured
- Cash timing differences may reflect intra-period movements
- Balance sheet identity failures may indicate complex structures (e.g., variable interest entities)
- A sample of 500 tickers yielded 1M+ errors, the probability of false positives is too high to impute.

---

### 8. validate_market_split_consistency

#### Error Addressed
**Inconsistency between raw market data and split-adjusted market data.** The relationship should be deterministic:
- `Adjusted_Price = Raw_Price × K`
- `Adjusted_Volume = Raw_Volume / K`

Where K is the cumulative split adjustment factor. When K_implied (from raw/adjusted) ≠ K_expected (from split events), the data is internally inconsistent.

#### Approach
1. Calculate daily split factor: `factor = denominator / numerator` (1.0 if no split)
2. Calculate cumulative K: `K_expected = cumulative_product(daily_factors)`
3. For each price column pair: `K_implied = adjusted / raw`
4. For volume: `K_implied = raw / adjusted` (inverse relationship)
5. Flag rows where `|K_implied - K_expected| > tolerance × |K_expected|`

#### Correction Method
Recalculate adjusted values from raw values using K_expected:
- **Prices:** `corrected_adjusted = raw × K_expected`
- **Volume:** `corrected_adjusted = raw / K_expected`

Validated column pairs:
| Raw Column | Adjusted Column | Relationship |
|------------|-----------------|--------------|
| m_open | m_open_split_adjusted | Price (×K) |
| m_high | m_high_split_adjusted | Price (×K) |
| m_low | m_low_split_adjusted | Price (×K) |
| m_close | m_close_split_adjusted | Price (×K) |
| m_vwap | m_vwap_split_adjusted | Price (×K) |
| m_volume | m_volume_split_adjusted | Volume (÷K) |

---

### Summary Matrix

| Function | Error Type | Detection Method | Correction Method |
|----------|------------|------------------|-------------------|
| sort_dates | Unsorted rows | Date comparison | Reorder in place |
| fill_negatives_fundamentals | Negative fundamentals | value < 0 | Forward-fill |
| fill_negatives_market | Negative prices | value < 0 | Cubic spline / forward-fill |
| zero_wipeout | Zero shares + positive volume | shares = 0 AND volume > 0 | Forward-fill |
| mkt_cap_scale_error | 10x jumps | value ≥ 10 × prior | Forward-fill (span-aware) |
| ohlc_integrity | OHLC constraint violations | H < max, L > min, VWAP bounds | Set to computed bounds |
| validate_financial_equivalencies | Accounting identity failures | |Total - Sum| > tolerance | Proportional scaling |
| validate_market_split_consistency | Split adjustment mismatch | K_implied ≠ K_expected | Recalculate from K_expected |

---

### Design Principles

1. **Full Audit Trail:** Every function returns a log of all modifications, enabling forensic analysis and regulatory compliance.

2. **Type Preservation:** Functions accept and return the same type (DataFrame ↔ DataFrame, LazyFrame ↔ LazyFrame).

3. **No Look-Forward Bias:** Corrections use only historical data, preserving validity for backtesting.

4. **Graceful Degradation:** When ideal corrections aren't possible (insufficient history, missing columns), functions fall back to simpler methods rather than failing.

5. **Configurable Tolerance:** Validation thresholds are parameterized, allowing adjustment for different data quality contexts.


## 03: Sanity check forensic log audit